In [ ]:
### Imports
%load_ext autoreload
%autoreload 2

# Append main folder
import sys
sys.path.append("../")

from glob import glob

import pykep as pk
import numpy as np
import matplotlib.pyplot as plt

dt = 36
starting_t = pk.epoch_from_string('2022-01-01 00:00:00.000')
iterations = 2400
end_t = pk.epoch(starting_t.mjd2000 + iterations * dt * pk.SEC2DAY)

In [ ]:
# Load test data
init_r = np.loadtxt("../data/pos.csv",delimiter=",")
init_v = np.loadtxt("../data/v.csv",delimiter=",")

In [ ]:
files = glob("../build/*.vtu")
file = "../build/output_10.vtu"

In [ ]:
from vtk import vtkXMLUnstructuredGridReader
from vtk.util import numpy_support as VN

reader = vtkXMLUnstructuredGridReader()
reader.SetFileName(file)
reader.Update()

data = reader.GetOutput()

v = VN.vtk_to_numpy(data.GetPointData().GetArray('velocity')).astype("double")
r = VN.vtk_to_numpy(data.GetPoints().GetData()).astype("double")

In [ ]:
v

In [ ]:
r

In [ ]:
from tqdm import tqdm
errs = [[],[],[],[],[],[]]

for v_p,r_p, init_v_p,init_r_p in tqdm(zip(v,r,init_v,init_r)):
    a,e,i,W,w,E = pk.ic2par(r_p,v_p, pk.MU_EARTH)
    oa,oe,oi,oW,ow,oE = pk.ic2par(init_r_p * 1000.0,init_v_p* 1000.0, pk.MU_EARTH)
    errs[0].append(abs(a-oa) / 1000.0)
    errs[1].append(abs(e-oe))
    errs[2].append(abs(i-oi))
    errs[3].append(abs(W-oW))
    errs[4].append(abs(w-ow))
    errs[5].append(abs(E-oE))

In [ ]:
for err,name in zip(errs, ["a","e","i","W","w","E"]):
    print(f"{name} Error - Mean. {np.mean(err):<10.6f} Min. {np.min(err):<8.6f} Max. {np.max(err):<8.6f}")